# Adattisztítás és integráció

A valós adathalmazok gyakran zajosak, hiányosak, avagy éppen redundáns információt vagy duplikátum egyedeket tartalmaznak. Ezért a tudásfeltárás folyamatában az adattisztítás és adatintegrálással kezdődik.

Az adattisztítás szerepe javítani az adatok minőségén azáltal, hogy kiszűri és eltávolítja az adatokban fellépő hibákat és inkonzisztenciákat.

Az [adattisztítás](https://hu.wikipedia.org/wiki/Adattiszt%C3%ADtás) során:
"
- felmérjük a hibákat
	- ellenőrizzük az adatfájl szerkezeti épségét
	- a zajt, felesleges információt tartalmazó mezőket javítjuk
	- felmérjük a hiányzó értékeket és amennyiben lehet ezeket pótoljuk
	- felmérjük az adatközlési és adatbeviteli hibákat
		- megvizsgáljuk az egyes változók eloszlását
			- az eloszlások szélein elhelyezkedő extrém értékeket ellenőrizzük
			- felmérjük, hogy az eloszlások megfelelnek-e az előzetes elvárásainknak, vannak-e nem várt sűrűsödések, ritkulások egyes értéktartományokban (például durva kerekítés vagy eltérő mértékegység használata az adatszolgáltatók egy részénél)
		-  megvizsgáljuk, hogy a változók közötti triviális összefüggések teljesülnek-e
-  a hibásnak tűnő adatokat felülvizsgáljuk, javítjuk 
".


# Feladatok

1. Az `egyetemek.txt` fájlból szűrjük ki az államokat és azon belül a városokat, melyben egyetemek találhatóak. Ha vannak duplikátumok, helytelen adatok (pl. számokat tartalmazó államnév), ezeket javítsuk. Vizsgáljuk meg az egyetemek eloszlását államok szerint. Melyik államban van a legtöbb, legkevesebb egyetem?


In [1]:
import pandas as pd


path = '/egyetemek.txt'
towns = []

with open(path, 'r') as file:
    items = file.readlines()
    states = list(filter(lambda x: ('[edit]' in x) and (all(c.isalpha() or c == '[' or c == ']' or c.isspace() for c in x)), items))

    statesNr = len(states)

    tchars = '[]().,?'

    for i, state in enumerate(states):
        start = items.index(state) + 1

        if i == statesNr-1: 
            end = len(items)
        else:
            end = items.index(states[i+1])

        tu = list(filter(lambda x: all(c.isalpha() or c.isspace() or c.isnumeric() or (c in tchars) for c in x), items[start:end]))

        pairs = map(lambda x: [state, x], tu)
        towns.extend(pairs)
    
df = pd.DataFrame(towns, columns=['State', 'Town'])
df.head()

def clean_town(item):
   return item[:item.find('(') - 1]

def clean_state(item):
   return item[:item.find('[')]

df['Town'] = df.Town.apply(clean_town)
df['State'] = df.State.apply(clean_state)
df.head()

df = df.drop_duplicates()
df.shape

(465, 2)

2. Bővítsük ki az adatbázisunkat egy oszloppal, mely tartalmazza az államok rövidítését is (pl. Texas - TX, California - CA stb.).  [Forrás](https://en.wikipedia.org/wiki/List_of_U.S._state_abbreviations).

In [2]:
us_state_abbrev = {
'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA', 'Colorado': 'CO',
'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID',
'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA',
'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ',
'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD',
'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA',
'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'}

df['State_abb'] = df['State'].map(us_state_abbrev).fillna(df['State'])

df

,State,Town,State_abb
0,Alabama,Auburn,AL
1,Alabama,Florence,AL
2,Alabama,Jacksonville,AL
3,Alabama,Livingston,AL
4,Alabama,Montevallo,AL
...,...,...,...
461,West Virginia,West Liberty,WV
462,Wisconsin,Appleton,WI
463,Wisconsin,Waukesha,WI
464,Wyoming,Laramie,WY


3. Az adatbázist integráljuk a [List of states and territories of the United States by population](https://en.wikipedia.org/wiki/List_of_states_and_territories_of_the_United_States_by_population) linken szereplő népszámlálási adatokkal és számoljuk ki államonként hány főre jut egy egyetemi város.

In [3]:
state_population = {"Alabama":4903185,"Alaska":731545,"Arizona":7278717,"Arkansas":3017825,"California":39512223,"Colorado":5758736,"Connecticut":3565287,
            "Delaware":973764,"Florida":21477737,"Georgia":10617423,"Hawaii":1415872,"Idaho":1787065,"Illinois":12671821,"Indiana":6732219,"Iowa":3155070,
            "Kansas":2913314,"Kentucky":4467673,"Louisiana":4648794,"Maine":1344212,"Maryland":6045680,"Massachusetts":6949503,"Michigan":9986857,
            "Minnesota":5639632,"Mississippi":2976149,"Missouri":6137428,"Montana":1068778,"Nebraska":1934408,"Nevada":3080156,"New Hampshire":1359711,
            "New Jersey":8882190,"New Mexico":2096829,"New York":19453561,"North Carolina":10488084,"North Dakota":762062,"Ohio":11689100,"Oklahoma":3956971,
            "Oregon":4217737,"Pennsylvania":12801989,"Rhode Island":1059361,"South Carolina":5148714,"South Dakota":884659,"Tennessee":6833174,"Texas":28995881,
            "Utah":3205958,"Vermont":623989,"Virginia":8535519,"Washington":7614893,"West Virginia":1792147,"Wisconsin":5822434,"Wyoming":578759}

df['Population'] = df['State'].map(state_population)
population = df.groupby(['State', 'State_abb','Population']).size().reset_index(name='University_Counts')
population['University/Citizen'] = round(population.Population/population.University_Counts,1)
population.head()

,State,State_abb,Population,University_Counts,University/Citizen
0,Alabama,AL,4903185,8,612898.1
1,Alaska,AK,731545,1,731545.0
2,Arizona,AZ,7278717,3,2426239.0
3,Arkansas,AR,3017825,8,377228.1
4,California,CA,39512223,25,1580488.9


4. Hasonlóan, a [List of U.S. states and territories by area](https://en.wikipedia.org/wiki/List_of_U.S._states_and_territories_by_area) linken szereplő területi adatok integrálásával, számoljuk ki államonként átlagban hány négyzetkilométerre jut egy egyetemi város.

In [4]:
allamter = {"Alabama":135767,"Alaska":135767,"Arizona":295234,"Arkansas":137732,"California":423967,"Colorado":269601,"Connecticut":14357,"Delaware":6446,
            "Florida":170312,"Georgia":153910,"Hawaii":28313,"Idaho":216443,"Illinois":149995,"Indiana":94326,"Iowa":145746,"Kansas":213100,"Kentucky":104656,
            "Louisiana":135659,"Maine":91633,"Maryland":32131,"Massachusetts":27336,"Michigan":250487,"Minnesota":225163,"Mississippi":125438,"Missouri":180540,
            "Montana":380831,"Nebraska":200330,"Nevada":286380,"New Hampshire":24214,"New Jersey":22591,"New Mexico":314917,"New York":141297,"North Carolina":139391,
            "North Dakota":183108,"Ohio":116098,"Oklahoma":181037,"Oregon":254799,"Pennsylvania":119280,"Rhode Island":4001,"South Carolina":82933,"South Dakota":199729,
            "Tennessee":109153,"Texas":695662,"Utah":219882,"Vermont":24906,"Virginia":110787,"Washington":184661,"West Virginia":62756,"Wisconsin":169635,"Wyoming":253335}
df['Area'] = df['State'].map(allamter)

area = df.groupby(["State","State_abb","Population","Area"]).size().reset_index(name="Counts")
area["Capital"] = round((area.Population/area.Counts),2)
area["Capital/km2"] = round((area.Area/area.Counts),2)
area

,State,State_abb,Population,Area,Counts,Capital,Capital/km2
0,Alabama,AL,4903185,135767,8,612898.12,16970.88
1,Alaska,AK,731545,135767,1,731545.00,135767.00
2,Arizona,AZ,7278717,295234,3,2426239.00,98411.33
3,Arkansas,AR,3017825,137732,8,377228.12,17216.50
4,California,CA,39512223,423967,25,1580488.92,16958.68
5,Colorado,CO,5758736,269601,8,719842.00,33700.12
6,Connecticut,CT,3565287,14357,7,509326.71,2051.00
7,Delaware,DE,973764,6446,2,486882.00,3223.00
8,Florida,FL,21477737,170312,10,2147773.70,17031.20
9,Georgia,GA,10617423,153910,13,816724.85,11839.23


5. A [példa](https://datasciencechalktalk.com/2019/09/28/analyzing-u-s-exports-with-plotly/), a kivitelt, exportmennyiséget ábrázolja az Amerikai Egyesült Államok térképen, államonként lebontva ezt. Készítsünk hasonló ábrákat az egyetemek abszolút, lakoság és terület szerinti eloszlásáról is. Az ábrákat exportáljuk kép formájában.
![](https://i.ibb.co/s1zdnLY/mapplot.png)

Ábra 1. [Térképen való ábrázolás](https://plotly.com/python/maps/) Plotly segítségével. [Forrás](https://datasciencechalktalk.com/2019/09/28/analyzing-u-s-exports-with-plotly/).

6. Számoljuk ki, hány egyetem van városonként, majd összesítve államonként. Ehhez az `egyetemek.txt` fájlban miután megkapunk egy várost, az utána következő kerek zárójelek között megszámoljuk, hány vesszővel elválasztott karakterlánc található. 
Pl. a következő sorban:	`Claremont (Claremont McKenna College, Pomona College, Harvey Mudd College, Scripps College, Pitzer College, Keck Graduate Institute, Claremont Graduate University)[5]` -> 6 vessző van a kerek zárójelek között, tehát 7 egyetem van a városban.    
Melyik államban és melyik városban van a legtöbb egyetem?

In [5]:
path = '/egyetemek.txt'

df2 = pd.DataFrame(columns=('State','City','UniversitiesCount'))

for line in path :
  if '[edit]' in line :
    state = re.sub("\[edit\]|[0-9]*",'',line)
    state = state.rstrip("\n")
  elif '(' in line:
    tmp = line.split('(')
    tmpcity = tmp[0]
    city = re.sub("\+|[0-9]*",'',tmpcity)
    tmpUni = tmp[1]
    tmpUni = re.sub("\)|[0-9]*|\[|\]",'',tmpUni)
    
    universities = tmpUni.split(',')
    u=len(universities)
    if len(state) > 0:
        row ={'State':state,'City':city,'UniversitiesCount':u}
        df2 = df2.append(row,ignore_index=True)
df2

,State,City,UniversitiesCount
